In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [38]:
import pandas as pd

In [71]:
import time

### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [62]:
KEYWORDS = ['python', 'интеллект', 'парсинг']

In [63]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [64]:
posts = soup.find_all('article', class_='post')

In [65]:
posts_filtred = []
for post in posts:
    for key in KEYWORDS:
        if key in post.text.lower():
            posts_filtred.append(post)
            break

In [79]:
def add_article(link):
    article = requests.get(link)
    time.sleep(0.3)
    article = BeautifulSoup(article.text, 'html.parser')
    article = article.find('div', class_ = 'post__text').text
    return article

In [169]:
habr_news = pd.DataFrame()
for post in posts_filtred:
    date = post.find('span', class_='post__time').text # Да, знаю, что надо бы преобразовать в нормальную дату, 
                                                       # но я уже 5 часов сижу с одним заданием... =(((
    title = post.find('a', class_='post__title_link').text
    link = post.find('a', class_ = 'post__title_link').get('href')   
    text = add_article(link)
    
    row = {'date': date, 'title': title, 'link' : link , 'text': text}
    habr_news = pd.concat([habr_news, pd.DataFrame([row])])
    
habr_news

,date,title,link,text
0,сегодня в 19:39,Как искусственный интеллект научили решать диф...,https://habr.com/ru/company/skillfactory/blog/...,"Сегодня, в преддверии старта нового потока кур..."
0,сегодня в 18:01,"Еще немного про C# 8.0, шпаргалка по Red Hat O...",https://habr.com/ru/company/redhatrussia/blog/...,"\n\r\nПрокачивайте скилы, читайте, смотрите, д..."
0,сегодня в 17:02,Внедряем оплату BTC куда угодно (Python),https://habr.com/ru/post/525638/,ПредысторияПолгода назад взялся за один проект...
0,сегодня в 16:54,Делаем поиск в веб-приложении с нуля,https://habr.com/ru/company/joom/blog/526550/,В статье «Делаем современное веб-приложение с ...
0,сегодня в 16:36,"21 канал на YouTube, где вы можете бесплатно и...",https://habr.com/ru/company/skillfactory/blog/...,\r\n Мы уже не раз делились в своем блоге поле...


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

- GROUP = 'netology'  
- TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [102]:
import json

In [160]:
from datetime import datetime

In [84]:
EMAILS = ['xxx@xxx.ru', 'yyy@y.com', 'xxx@x.ru', 'hommka007@yandex.ru']

In [204]:
def check_emails(EMAILS):
    
    check_emails_df = pd.DataFrame()
    
    for email in EMAILS:
        URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

        data = {
            'emailAddresses': [email]
                }

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
            'Vaar-Header-App-Product': 'hackcheck-web-avast',
            'Vaar-Version': '0'
                   }

        req = requests.post(URL, json = data, headers = headers).json()
        time.sleep(0.3)
        
        links = []
        descriptiones = []
        dates = []
        
        for x_1_k, x_1_v in req['breaches'].items():
            links.append(x_1_v['site'])
            descriptiones.append(x_1_v['description'])
            dates.append(x_1_v['publishDate'])
        
        dates_e = []

        for date in dates:
            date_p = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
            dates_e.append(datetime.strftime(date_p, '%Y/%m/%d'))
           
        row = {'email': email, 'date': dates_e, 'source' : links , 'description': descriptiones}
        check_emails_df = pd.concat([check_emails_df, pd.DataFrame(row)])
        
    return check_emails_df
    

In [205]:
check_emails(EMAILS)

,email,date,source,description
0,xxx@xxx.ru,2018/01/07,fotoboom.com,"In July 2017, Fotoboom's user database was all..."
1,xxx@xxx.ru,2018/04/05,fotoboom.com,"In July 2017, Fotoboom's database was allegedl..."
2,xxx@xxx.ru,2016/10/24,000webhost.com,An anonymous criminal was able to exploit an o...
3,xxx@xxx.ru,2016/11/01,qip.ru,"In 2011, Russian instant messaging service pro..."
4,xxx@xxx.ru,2018/06/19,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
5,xxx@xxx.ru,2018/03/03,tunnel.de,"At an unconfirmed date, The Tunnel Club's data..."
6,xxx@xxx.ru,2017/11/04,myheritage.com,"In October 2017, a customer database belonging..."
7,xxx@xxx.ru,2017/02/14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
8,xxx@xxx.ru,2018/03/29,oasiscollectors.com,"At an unconfirmed date, Oasis Collectors' data..."
9,xxx@xxx.ru,2019/07/25,www.oasiscollectors.com,"In February 2018, UK rock fan site Oasis Colle..."
